# Test client

In [27]:
import yaml
def update_username(username):
    path = 'client/feature_store.yaml'
    with open(path, 'r') as file:
        config = yaml.safe_load(file) or {}
    config['auth']['username'] = username
    with open(path, 'w') as file:
        yaml.safe_dump(config, file, default_flow_style=False)

# Update test user
Use one of `reader`, `writer`, `batch_admin` or `admin` (password is fixed) as the current `username`.

In [86]:
username = 'admin'
update_username(username)

In [87]:
!cat client/feature_store.yaml

auth:
  auth_discovery_url: http://0.0.0.0:9999/realms/rbac_example/.well-known/openid-configuration
  auth_server_url: http://0.0.0.0:9999
  client_id: app
  client_secret: eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8
  password: password
  realm: rbac_example
  type: oidc
  username: admin
entity_key_serialization_version: 2
offline_store:
  host: localhost
  port: 8815
  type: remote
online_store:
  path: http://localhost:6566
  type: remote
project: rbac
registry:
  path: localhost:6570
  registry_type: remote


## Updating logger
The following is needed to log in the notebook the output the messages logged by th Feast application.

In [88]:
import logging
import sys
from io import StringIO
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger()

## Setup Feast client
Initialize the Feast store from the [client configuration](./client/feature_store.yaml)

In [89]:
from feast.feature_store import FeatureStore

In [90]:
store = FeatureStore(repo_path="client")

## Basic validation
Verify the authorization config and run some GET APIs on the registry.

In [91]:
print(f"Authorization config is: {store.config.auth}")

Authorization config is: {'auth_discovery_url': 'http://0.0.0.0:9999/realms/rbac_example/.well-known/openid-configuration', 'auth_server_url': 'http://0.0.0.0:9999', 'client_id': 'app', 'client_secret': 'eFGTp6ZJ0tARKYPbdMKUZ1J8PJNWfXR8', 'password': 'password', 'realm': 'rbac_example', 'type': 'oidc', 'username': 'admin'}


In [92]:
for e in store.list_entities():
    print(f"Entity found {e.name}")

2024-08-09 10:12:55,608 - Intercepted the grpc api method /feast.registry.RegistryServer/ListEntities call to inject Authorization header token. 


Entity found driver


In [93]:
for fv in store.list_all_feature_views():
    print(f"FeatureView found {fv.name} of type {type(fv).__name__}")

2024-08-09 10:12:57,920 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:12:57,921 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 
2024-08-09 10:12:58,307 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:12:58,602 - Intercepted the grpc api method /feast.registry.RegistryServer/ListOnDemandFeatureViews call to inject Authorization header token. 


FeatureView found driver_hourly_stats of type FeatureView
FeatureView found driver_hourly_stats_fresh of type FeatureView
FeatureView found transformed_conv_rate_fresh of type OnDemandFeatureView
FeatureView found transformed_conv_rate of type OnDemandFeatureView


In [94]:
for fs in store.list_feature_services():
    print(f"FeatureService found {fs.name} of type {type(fs).__name__}")

2024-08-09 10:13:01,877 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureServices call to inject Authorization header token. 


FeatureService found driver_activity_v2 of type FeatureService
FeatureService found driver_activity_v1 of type FeatureService
FeatureService found driver_activity_v3 of type FeatureService


In [95]:
!feast -c client permissions list

NAME                    TYPES                WITH_SUBCLASS    NAME_PATTERN    ACTIONS        ROLES
read_permission         FeatureView          True                             READ           reader
                        OnDemandFeatureView
                        BatchFeatureView
                        StreamFeatureView
                        Entity
                        FeatureService
                        DataSource
                        ValidationReference
                        SavedDataset
                        Permission
write_fresh_permission  FeatureView          True             .*_fresh        WRITE_ONLINE   fresh_writer
offline_permission      FeatureView          True                             CREATE         batch_admin
                        OnDemandFeatureView                                   READ
                        FeatureService                                        UPDATE
                                                                          

## Validating with test_workflow.py
The following test functions were copied from the `test_workflow.py` template but we added `try` blocks to print only 
the relevant error messages, since we expect to receive errors from the permission enforcement modules.

In [96]:
import subprocess
from datetime import datetime

import pandas as pd

from feast import FeatureStore
from feast.data_source import PushMode

def fetch_historical_features_entity_df(store: FeatureStore, for_batch_scoring: bool):
    # Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for more details on how to retrieve
    # for all entities in the offline store instead
    entity_df = pd.DataFrame.from_dict(
        {
            # entity's join key -> entity values
            "driver_id": [1001, 1002, 1003],
            # "event_timestamp" (reserved key) -> timestamps
            "event_timestamp": [
                datetime(2021, 4, 12, 10, 59, 42),
                datetime(2021, 4, 12, 8, 12, 10),
                datetime(2021, 4, 12, 16, 40, 26),
            ],
            # (optional) label name -> label values. Feast does not process these
            "label_driver_reported_satisfaction": [1, 5, 3],
            # values we're using for an on-demand transformation
            "val_to_add": [1, 2, 3],
            "val_to_add_2": [10, 20, 30],
        }
    )
    # For batch scoring, we want the latest timestamps
    if for_batch_scoring:
        entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)

    try:
        training_df = store.get_historical_features(
            entity_df=entity_df,
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
                "transformed_conv_rate:conv_rate_plus_val1",
                "transformed_conv_rate:conv_rate_plus_val2",
            ],
        ).to_df()
        print(training_df.head())
    except Exception as e:
        print(f"Failed to run `store.get_historical_features`: {e}")


def fetch_online_features(store, source: str = ""):
    entity_rows = [
        # {join_key: entity_value}
        {
            "driver_id": 1001,
            "val_to_add": 1000,
            "val_to_add_2": 2000,
        },
        {
            "driver_id": 1002,
            "val_to_add": 1001,
            "val_to_add_2": 2002,
        },
    ]
    if source == "feature_service":
        try:
            features_to_fetch = store.get_feature_service("driver_activity_v1")
        except Exception as e:
            print(f"Failed to run `store.get_feature_service`: {e}")
    elif source == "push":
        try:
            features_to_fetch = store.get_feature_service("driver_activity_v3")
        except Exception as e:
            print(f"Failed to run `store.get_feature_service`: {e}")
    else:
        features_to_fetch = [
            "driver_hourly_stats:acc_rate",
            "transformed_conv_rate:conv_rate_plus_val1",
            "transformed_conv_rate:conv_rate_plus_val2",
        ]
    try:
        returned_features = store.get_online_features(
            features=features_to_fetch,
            entity_rows=entity_rows,
        ).to_dict()
        for key, value in sorted(returned_features.items()):
            print(key, " : ", value)
    except Exception as e:
        print(f"Failed to run `store.get_online_features`: {e}")

In [97]:
store = FeatureStore(repo_path="client")

### Historical features

In [98]:
print("\n--- Historical features for training ---")
fetch_historical_features_entity_df(store, for_batch_scoring=False)

print("\n--- Historical features for batch scoring ---")
fetch_historical_features_entity_df(store, for_batch_scoring=True)

2024-08-09 10:13:15,551 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:15,552 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 



--- Historical features for training ---


2024-08-09 10:13:15,893 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:16,196 - Intercepted the grpc api method /feast.registry.RegistryServer/ListOnDemandFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:16,511 - Connecting FlightClient at grpc://localhost:8815
2024-08-09 10:13:16,924 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:16,925 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 


   driver_id           event_timestamp  label_driver_reported_satisfaction  \
0       1001 2021-04-12 10:59:42+00:00                                   1   
1       1002 2021-04-12 08:12:10+00:00                                   5   
2       1003 2021-04-12 16:40:26+00:00                                   3   

   val_to_add  val_to_add_2  conv_rate  acc_rate  avg_daily_trips  \
0           1            10   0.928541  0.695293              818   
1           2            20   0.491019  0.712657              233   
2           3            30   0.504265  0.638185              738   

   conv_rate_plus_val1  conv_rate_plus_val2  
0             1.928541            10.928541  
1             2.491019            20.491019  
2             3.504265            30.504265  

--- Historical features for batch scoring ---


2024-08-09 10:13:17,229 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:17,585 - Intercepted the grpc api method /feast.registry.RegistryServer/ListOnDemandFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:17,937 - Connecting FlightClient at grpc://localhost:8815


   driver_id                  event_timestamp  \
0       1002 2024-08-09 08:13:16.924405+00:00   
1       1001 2024-08-09 08:13:16.924405+00:00   
2       1003 2024-08-09 08:13:16.924405+00:00   

   label_driver_reported_satisfaction  val_to_add  val_to_add_2  conv_rate  \
0                                   5           2            20   0.709591   
1                                   1           1            10   0.304952   
2                                   3           3            30   0.062595   

   acc_rate  avg_daily_trips  conv_rate_plus_val1  conv_rate_plus_val2  
0  0.446933              543             2.709591            20.709591  
1  0.228232              765             1.304952            10.304952  
2  0.787014              147             3.062595            30.062595  


### Materialization

In [99]:
print("\n--- Load features into online store ---")
try:
    store.materialize_incremental(end_date=datetime.now())
except Exception as e:
    print(f"Failed to run `store.materialize_incremental`: {e}")

2024-08-09 10:13:20,397 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:20,398 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 



--- Load features into online store ---


2024-08-09 10:13:20,731 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:21,034 - Intercepted the grpc api method /feast.registry.RegistryServer/GetEntity call to inject Authorization header token. 


Materializing 2 feature views to 2024-08-09 10:13:20+02:00 into the remote online store.

driver_hourly_stats from 2024-08-08 19:25:13+02:00 to 2024-08-09 10:13:20+02:00:


2024-08-09 10:13:21,345 - Connecting FlightClient at grpc://localhost:8815
0it [00:00, ?it/s]
2024-08-09 10:13:23,864 - Intercepted the grpc api method /feast.registry.RegistryServer/ApplyMaterialization call to inject Authorization header token. 
2024-08-09 10:13:24,183 - Intercepted the grpc api method /feast.registry.RegistryServer/GetEntity call to inject Authorization header token. 


driver_hourly_stats_fresh from 2024-08-08 19:25:13+02:00 to 2024-08-09 12:13:20+02:00:


2024-08-09 10:13:24,484 - Connecting FlightClient at grpc://localhost:8815
0it [00:00, ?it/s]
2024-08-09 10:13:24,848 - Intercepted the grpc api method /feast.registry.RegistryServer/ApplyMaterialization call to inject Authorization header token. 


### Online features

In [100]:
print("\n--- Online features ---")
fetch_online_features(store)

print("\n--- Online features retrieved (instead) through a feature service---")
fetch_online_features(store, source="feature_service")

print(
    "\n--- Online features retrieved (using feature service v3, which uses a feature view with a push source---"
)
fetch_online_features(store, source="push")

2024-08-09 10:13:26,503 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:26,504 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 



--- Online features ---


2024-08-09 10:13:26,844 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:27,139 - Intercepted the grpc api method /feast.registry.RegistryServer/ListOnDemandFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:27,588 - Intercepted the grpc api method /feast.registry.RegistryServer/ListEntities call to inject Authorization header token. 
2024-08-09 10:13:27,943 - Intercepted the grpc api method /feast.registry.RegistryServer/GetEntity call to inject Authorization header token. 
2024-08-09 10:13:28,262 - Intercepted the grpc api method /feast.registry.RegistryServer/GetEntity call to inject Authorization header token. 
2024-08-09 10:13:28,928 - Intercepted the grpc api method /feast.registry.RegistryServer/GetFeatureService call to inject Authorization header token. 


acc_rate  :  [0.22823210060596466, 0.44693297147750854]
conv_rate_plus_val1  :  [1000.3049515485764, 1001.7095908522606]
conv_rate_plus_val2  :  [2000.3049515485764, 2002.7095908522606]
driver_id  :  [1001, 1002]

--- Online features retrieved (instead) through a feature service---


2024-08-09 10:13:29,223 - Intercepted the grpc api method /feast.registry.RegistryServer/GetFeatureService call to inject Authorization header token. 
2024-08-09 10:13:29,513 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:29,513 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:29,822 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:30,166 - Intercepted the grpc api method /feast.registry.RegistryServer/ListOnDemandFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:30,517 - Intercepted the grpc api method /feast.registry.RegistryServer/ListEntities call to inject Authorization header token. 
2024-08-09 10:13:30,820 - Intercepted the g

conv_rate  :  [0.304951548576355, 0.7095908522605896]
conv_rate_plus_val1  :  [1000.3049515485764, 1001.7095908522606]
conv_rate_plus_val2  :  [2000.3049515485764, 2002.7095908522606]
driver_id  :  [1001, 1002]

--- Online features retrieved (using feature service v3, which uses a feature view with a push source---


2024-08-09 10:13:32,117 - Intercepted the grpc api method /feast.registry.RegistryServer/GetFeatureService call to inject Authorization header token. 
2024-08-09 10:13:32,413 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:32,414 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:32,736 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:33,029 - Intercepted the grpc api method /feast.registry.RegistryServer/ListOnDemandFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:33,322 - Intercepted the grpc api method /feast.registry.RegistryServer/ListEntities call to inject Authorization header token. 
2024-08-09 10:13:33,611 - Intercepted the g

acc_rate  :  [0.22823210060596466, 0.44693297147750854]
avg_daily_trips  :  [765, 543]
conv_rate  :  [0.304951548576355, 0.7095908522605896]
conv_rate_plus_val1  :  [1000.3049515485764, 1001.7095908522606]
conv_rate_plus_val2  :  [2000.3049515485764, 2002.7095908522606]
driver_id  :  [1001, 1002]


### Stream push

In [101]:
print("\n--- Simulate a stream event ingestion of the hourly stats df ---")
event_df = pd.DataFrame.from_dict(
    {
        "driver_id": [1001],
        "event_timestamp": [
            datetime.now(),
        ],
        "created": [
            datetime.now(),
        ],
        "conv_rate": [1.0],
        "acc_rate": [1.0],
        "avg_daily_trips": [1000],
    }
)
print(event_df)
try:
    store.push("driver_stats_push_source", event_df, to=PushMode.ONLINE_AND_OFFLINE)
    print("\n--- Online features again with updated values from a stream push---")
    fetch_online_features(store, source="push")
except Exception as e:
    print(f"Failed to run `store.push`: {e}")

2024-08-09 10:13:34,231 - list_feature_views will make breaking changes. Please use list_batch_feature_views instead. list_feature_views will behave like list_all_feature_views in the future.
2024-08-09 10:13:34,232 - _list_feature_views will make breaking changes. Please use _list_batch_feature_views instead. _list_feature_views will behave like _list_all_feature_views in the future.
2024-08-09 10:13:34,233 - Intercepted the grpc api method /feast.registry.RegistryServer/ListFeatureViews call to inject Authorization header token. 



--- Simulate a stream event ingestion of the hourly stats df ---
   driver_id            event_timestamp                    created  conv_rate  \
0       1001 2024-08-09 10:13:34.227334 2024-08-09 10:13:34.227339        1.0   

   acc_rate  avg_daily_trips  
0       1.0             1000  


2024-08-09 10:13:34,534 - Intercepted the grpc api method /feast.registry.RegistryServer/ListStreamFeatureViews call to inject Authorization header token. 
2024-08-09 10:13:34,838 - Intercepted the grpc api method /feast.registry.RegistryServer/GetStreamFeatureView call to inject Authorization header token. 


Failed to run `store.push`: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "Exception calling application: Feature view driver_hourly_stats_fresh does not exist in project rbac"
	debug_error_string = "UNKNOWN:Error received from peer  {grpc_message:"Exception calling application: Feature view driver_hourly_stats_fresh does not exist in project rbac", grpc_status:2, created_time:"2024-08-09T10:13:35.137923+02:00"}"
>


**Note** If you see the following error, it is likely due to the issue [#4392: Remote registry client does not map application errors](https://github.com/feast-dev/feast/issues/4392):
```
Feature view driver_hourly_stats_fresh does not exist in project rbac
```